In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.metrics import confusion_matrix, classification_report, precision_score

In [2]:
path_csv_file = "C:/Users/husey/PycharmProjects/pythonProject12/IntToNLP_GRP_9/latest_hukums_with_classes_csv_file1.csv"

df = pd.read_csv(path_csv_file)
df.head()

,Unnamed: 0,Unnamed: 0.1,ictihat,Suç,YeniSuclar
0,0,0,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",hakaret,Şerefe karşı suçlar
1,1,1,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",tehdit,Hürriyete karşı suçlar
2,2,2,"HÜKÜMLER : Mahkumiyet, beraat, hükmün açıkl...",yaralama,Vücut dokunulmazlığına karşı suçlar
3,3,3,HÜKÜMLER : Beraat Yerel Mahkemece bozma ...,tehdit,Hürriyete karşı suçlar
4,4,4,HÜKÜMLER : Beraat Yerel Mahkemece bozma ...,hakaret,Şerefe karşı suçlar


In [4]:
label_map = {}
count = 0
for label in df.YeniSuclar.unique():
    label_map[label] = count
    count += 1
df['NUM_LABEL'] = df.YeniSuclar.map(label_map)
#print(label_map)
df.YeniSuclar.value_counts()

Mal varlığına karşı suçlar                                    3137
Hürriyete karşı suçlar                                        2883
Vücut dokunulmazlığına karşı suçlar                           2684
others                                                        2364
Kamu güvenine karşı suçlar                                    1993
Şerefe karşı suçlar                                           1317
Kamunun sağlığına karşı suçlar                                1216
Kamu idaresinin güvenilirliğine ve işleyişine karşı suçlar     837
Anayasal düzene ve bu düzenin işleyişine karşı suçlar          779
Cinsel dokunulmazlığa karşı suçlar                             607
Adliyeye karşı suçlar                                          401
Hayata karşı suçlar                                            254
Genel tehlike yaratan suçlar                                   249
Çevreye karşı suçlar                                           171
Ekonomi, sanayi ve ticarete ilişkin suçlar                    

In [6]:
texts = df.ictihat
label_nums = df.NUM_LABEL

X_train, X_test, y_train, y_test = train_test_split(texts, label_nums, test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(15424,)
(3856,)
(15424,)
(3856,)


In [7]:
def clean_text(text):
    alphabetic_only = [word for word in text.split() if word.isalpha()]
    lower_case_only = [word.lower() for word in alphabetic_only]
    stopwords_tr = set(stopwords.words("turkish"))
    return [word for word in lower_case_only if word not in stopwords_tr]


In [8]:
vect = CountVectorizer(analyzer='word', tokenizer=clean_text)
X_train_dtm = vect.fit_transform(X_train.astype("U"))
X_train_dtm

<15424x23472 sparse matrix of type '<class 'numpy.int64'>'
	with 1547633 stored elements in Compressed Sparse Row format>

In [9]:
X_test_dtm = vect.transform(X_test.astype("U"))
X_test_dtm

<3856x23472 sparse matrix of type '<class 'numpy.int64'>'
	with 380648 stored elements in Compressed Sparse Row format>

In [10]:
"""
vect2 = TfidfTransformer()
X_train_tfidf = vect2.fit_transform(X_train_dtm.astype("U"))
X_train_tfidf
"""

'\nvect2 = TfidfTransformer()\nX_train_tfidf = vect2.fit_transform(X_train_dtm.astype("U"))\nX_train_tfidf\n'

In [11]:
"""
X_test_tfidf = vect2.transform(X_test_dtm.astype("U"))
X_test_tfidf
"""

'\nX_test_tfidf = vect2.transform(X_test_dtm.astype("U"))\nX_test_tfidf\n'

In [12]:
model = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
model.fit(X_train_dtm, y_train)

SVC(gamma='auto', kernel='linear')

In [16]:
y_pred_class = model.predict(X_test_dtm)
print('********************************SVM********************************')
print(metrics.accuracy_score(y_test, y_pred_class))
print('********************************Confusion Matrix********************************')
print(confusion_matrix(y_test, y_pred_class))
print('********************************Precision score macro********************************')
print(precision_score(y_test, y_pred_class, average='macro', zero_division=1))
print('********************************Precision score micro********************************')
print(precision_score(y_test, y_pred_class, average='micro', zero_division=1))
print('********************************Precision score weighted********************************')
print(precision_score(y_test, y_pred_class, average='weighted', zero_division=1))
print('********************************Precision score null********************************')
print(precision_score(y_test, y_pred_class, average=None, zero_division=1))
print('********************************Classification report********************************')
print(classification_report(y_test, y_pred_class, zero_division=1))

********************************SVM********************************
0.578838174273859
********************************Confusion Matrix********************************
[[ 96  83  36   6   7  31   0   1   2   0   1   0   0   0   0   1   0   0
    0]
 [105 232  62  32  79   4   0  14  10   0  32   1   0   0   0   2   1   0
    4]
 [ 39  61 367  24  18   4   0  16   3   4   6   0   0   0   1   0   0   0
    2]
 [ 17  45  20 190  72  14  15  17  10  14   8   1   1   1  39   3   1   0
    0]
 [  7  77  14  54 458  10   0   1   0   4   1   0   3   0  25   0   0   0
    0]
 [ 28   6   6  19  30  74   0   0   0   0   0   3   0   0   3   0   0   0
    1]
 [  0   0   0   4   0   0 230   0   0   0   0   0   0   0   5   0   0   0
    0]
 [  2  18  15  19   1   2   0  82   6   4   0   0   0   2   1   0   0   0
    0]
 [  2   7   0  10   4   1   1   9  44   0   0   0   0   0   2   0   0   0
    0]
 [  0   1   2  16   2   0   0   0   0  27   0   0   0   0   0   0   0   0
    0]
 [  3  28   2  10   0  